# Colour viewer for HOYS lightcurves

* ***Shift + Enter on a code cell to run it and advance to the next cell.***

* ***If changing sliders, rerun the code block below the sliders***

* ***Plot windows are interactive, can use mouse to select and zoom into ranges***

In [1]:
#load required modules
import os
import pandas as pd
import astropy
import numpy
import math
from astropy.time import Time
from astropy.timeseries import LombScargle
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

### List available lightcurve data and select from dropdown list
* Upload your .csv file to '/light_curve_csv_files'

In [2]:
#read HOYS csv file
lc_folder = "./light_curve_csv_files/"
star_list=os.listdir(lc_folder)

lc_select = widgets.Dropdown(
    options=sorted(star_list),
    description='Select a light curve file:',
    style = {'description_width': 'initial'}
)
display(lc_select)

Dropdown(description='Select a light curve file:', options=('.DS_Store', 'lightcurve_2MASSJ20494917+4410462.tx…

### Display head and tail of csv file containing all of the data

In [9]:
print('selected datafile:',lc_select.value)
lc_data = pd.read_csv(os.path.join(lc_folder,lc_select.value),comment='#',delimiter=' ')
pd.set_option('display.max_columns', None)
#view file head and tail
lc_data

selected datafile: lightcurve_V2492Cyg.txt


,id,calibrated_magnitude,calibrated_error,magnitude_rms_error,x,y,alpha_j2000,delta_j2000,fwhm_world,flags,magnitude,observation_id,filter,original_filter,date,user_id,device_id,target,fits_id
0,21811158,11.9285,0.026363,0.0008,2456.430908,1078.546997,312.859521,44.088917,0.001010,0.0,12.6169,1558,I,I-Band,2.457768e+06,7,2,[118] IC5070_incl_201_V2492Cyg,1458
1,39822543,15.2650,0.059730,0.0083,2450.033691,962.726379,312.858921,44.088933,0.000953,0.0,15.7192,3129,I,I-Band,2.458054e+06,7,2,[118] IC5070_incl_201_V2492Cyg,2833
2,65082718,14.5882,0.049483,0.0128,301.396515,1325.128174,312.858423,44.088937,0.001431,0.0,12.8108,5048,R,-Rc-,2.458332e+06,33,56,[118] IC5070_incl_201_V2492Cyg,4602
3,41209812,15.7885,0.091734,0.0149,2547.287842,953.002686,312.859929,44.088940,0.000911,0.0,15.3581,3208,R,Red,2.458063e+06,7,2,[118] IC5070_incl_201_V2492Cyg,2912
4,20283949,12.2721,0.021319,0.0007,2456.248291,1023.680176,312.858596,44.088941,0.000860,0.0,12.7532,1441,I,I-Band,2.457723e+06,7,2,[118] IC5070_incl_201_V2492Cyg,1344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,201693094,16.5990,0.111832,0.0147,2356.322754,507.941010,312.859521,44.090543,0.000900,0.0,27.4031,34003,B,TB,2.459110e+06,63,168,[118] IC5070_incl_201_V2492Cyg,33736
2991,207353001,18.8418,0.245300,0.0801,2248.417480,412.260101,312.859483,44.090544,0.000548,0.0,17.3834,36312,B,TB,2.459150e+06,63,168,[118] IC5070_incl_201_V2492Cyg,35239
2992,331061928,18.4307,0.254098,0.1220,26.049500,750.829712,312.860016,44.090566,-0.000142,0.0,28.0529,56699,R,Red,2.459513e+06,58,143,[118] IC5070_incl_201_V2492Cyg,55951
2993,24412203,13.6913,0.011990,0.0026,1713.931641,3132.683105,312.859169,44.090574,0.001435,0.0,13.5566,1989,R,Red,2.457835e+06,7,2,[118] IC5070_incl_201_V2492Cyg,1835


In [10]:
filter_list=numpy.unique(lc_data['filter'])

# filter_select = widgets.SelectMultiple(
#     options=sorted(filter_list),
#     value=['B','V','R','I'],
#     description='Select a filter',
#     style = {'description_width': 'initial'}
# )
# display(filter_select)

jd=Time(lc_data.date,format='jd') 
mjd=jd.mjd #plot mjd instead of jd
date_range_sel=widgets.FloatRangeSlider(
    value=[min(mjd),max(mjd)],
    min=min(mjd),
    max=max(mjd),
    step=0.1,
    description='Date range [mjd]:',
    readout_format='.1f',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}

)
display(date_range_sel)

cal_error_sel=widgets.FloatSlider(
    value=0.25,
    min=0.1,
    max=0.4,
    step=0.01,
    description='Calibrated error [mag]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(cal_error_sel)

med_window_sel=widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Median filter window [days]:',
    layout={'width': '400px'},
    style = {'description_width': 'initial'}
)
display(med_window_sel)

FloatRangeSlider(value=(57274.95849000011, 59634.11098379968), description='Date range [mjd]:', layout=Layout(…

FloatSlider(value=0.25, description='Calibrated error [mag]:', layout=Layout(width='600px'), max=0.4, min=0.1,…

IntSlider(value=1, description='Median filter window [days]:', layout=Layout(width='400px'), max=10, min=1, st…

In [11]:
#select data from specifed filter and remove bad data
#band=filter_select.value
band=['B','V','R','I']
cal_error=cal_error_sel.value
med_window=med_window_sel.value
date_range=Time(date_range_sel.value,format='mjd').jd

filtered_data=lc_data[(lc_data['filter'].isin(band)) & (lc_data['flags'] <= 4) & (lc_data['calibrated_error'] < cal_error) & (lc_data['date'] > date_range[0]) & (lc_data['date'] < date_range[1]) & (lc_data['calibrated_magnitude'] > 0.0) & (lc_data['calibrated_error'] > 0.0) & (lc_data['fwhm_world'] > 0.0) & (lc_data['fwhm_world'] < 9.0/3600.0)]
filtered_data.reset_index(inplace=True,drop=True)
#filtered_data['utc']=pd.to_datetime(filtered_data['date'],unit='D', origin='julian')

#start=Time.now()

# slow method in pandas
# for date in filtered_data.index:
#     med_mask=(abs(filtered_data['date'][date] - filtered_data['date']) < 1)
#     for fil in band:
#         filtered_data.loc[med_mask,fil]=numpy.median(filtered_data[med_mask]['calibrated_magnitude'][(filtered_data[med_mask]['filter']==fil)])

#fast method with numpy arrays
date=numpy.array(filtered_data['date'])
filt=numpy.array(filtered_data['filter'])
mag=numpy.array(filtered_data['calibrated_magnitude'])
dt2=numpy.dtype([('B',numpy.float32),('V',numpy.float32),('R',numpy.float32),('I',numpy.float32),
                ('Be',numpy.float32),('Ve',numpy.float32),('Re',numpy.float32),('Ie',numpy.float32)])
colours=numpy.empty(mag.shape,dtype=dt2)
colours[:]=numpy.nan
for i in range(0,len(filtered_data)):
    for f in band:
        check = numpy.where( (filt == f) &  (numpy.abs(date[i] - date) < med_window) )
        if len(check[0]>0):
            colours[i][f]=numpy.median(mag[check[0]])
            colours[i][f+'e'] = numpy.nanstd(mag[check[0]])/numpy.sqrt(float(len(mag[check[0]])))

        
filtered_data=pd.concat((filtered_data,pd.DataFrame(colours)),axis=1)

display(filtered_data.sort_values(by='date'))

#end=Time.now()
#print((end-start).sec)

,id,calibrated_magnitude,calibrated_error,magnitude_rms_error,x,y,alpha_j2000,delta_j2000,fwhm_world,flags,magnitude,observation_id,filter,original_filter,date,user_id,device_id,target,fits_id,B,V,R,I,Be,Ve,Re,Ie
55,2274778,15.7722,0.059837,0.0080,2414.250000,1009.126709,312.859866,44.089256,0.000949,0.0,15.3390,89,V,Visual,2.457275e+06,7,2,[118] IC5070_incl_201_V2492Cyg,93,NaN,15.772200,14.6408,NaN,NaN,0.000000,0.0,NaN
867,163532,14.6408,0.031629,0.0037,2416.836670,1009.342407,312.859358,44.089662,0.000993,2.0,13.9247,94,R,Red,2.457275e+06,7,2,[118] IC5070_incl_201_V2492Cyg,94,NaN,15.772200,14.6408,NaN,NaN,0.000000,0.0,NaN
1082,4008709,16.6896,0.101975,0.0623,1751.372192,3089.029053,312.859590,44.089722,0.001939,0.0,17.5090,318,B,Blue,2.457463e+06,7,2,[118] IC5070_incl_201_V2492Cyg,294,16.6896,15.045300,14.0682,13.1314,0.0,0.000000,0.0,0.0
217,4051179,15.0453,0.016443,0.0139,1747.037476,3095.377686,312.859105,44.089453,0.001394,0.0,15.3557,317,V,Visual,2.457463e+06,7,2,[118] IC5070_incl_201_V2492Cyg,298,16.6896,15.045300,14.0682,13.1314,0.0,0.000000,0.0,0.0
366,4032045,14.0682,0.015294,0.0062,1743.697876,3095.330078,312.859182,44.089505,0.001355,0.0,13.9481,315,R,Red,2.457463e+06,7,2,[118] IC5070_incl_201_V2492Cyg,296,16.6896,15.045300,14.0682,13.1314,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,363525035,15.6614,0.135884,0.0939,1734.394653,3132.896729,312.859123,44.089490,0.001913,0.0,16.5970,63272,R,Red,2.459616e+06,7,2,[118] IC5070_incl_201_V2492Cyg,62898,NaN,NaN,15.6614,14.5815,NaN,NaN,0.0,0.0
875,368405206,17.3556,0.236174,0.1476,1712.841553,3002.922852,312.858973,44.089663,0.001476,0.0,18.4719,64382,V,Visual,2.459634e+06,7,2,[118] IC5070_incl_201_V2492Cyg,63325,NaN,17.015949,15.5009,14.4214,NaN,0.240169,0.0,0.0
544,368414318,15.5009,0.094170,0.0402,1710.918579,3007.699951,312.859262,44.089564,0.001725,0.0,16.6279,64384,R,Red,2.459634e+06,7,2,[118] IC5070_incl_201_V2492Cyg,63327,NaN,17.015949,15.5009,14.4214,NaN,0.240169,0.0,0.0
479,368429870,14.4214,0.052791,0.0183,1710.975830,3008.107666,312.859312,44.089545,0.001474,0.0,16.3264,64385,I,I-Band,2.459634e+06,7,2,[118] IC5070_incl_201_V2492Cyg,63328,NaN,17.015949,15.5009,14.4214,NaN,0.240169,0.0,0.0


### Lightcurve plot

In [12]:
def plot_lightcurve(mag_range_select=[min(filtered_data['calibrated_magnitude']),max(filtered_data['calibrated_magnitude'])],error_bars=False):
    global mag_data
    mag_range=mag_range_select
    mag_data=filtered_data[(filtered_data['calibrated_magnitude'] > mag_range[0]) & (filtered_data['calibrated_magnitude'] < mag_range[1])]

    jd=Time(mag_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd
    
    color_discrete_map = {'B': 'blue', 'V': 'green', 'R': 'red', 'I':'black'}

    #plot lightcurve
    if error_bars==True:
        fig1=px.scatter(mag_data,x=mjd,y=mag_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'},
                       error_y=mag_data.calibrated_error)
    else:
        fig1=px.scatter(mag_data,x=mjd,y=mag_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'})
    fig1['layout']['yaxis_title']='Magnitude [mag]'
    fig1['layout']['yaxis']['autorange'] = "reversed"
    fig1['layout']['xaxis']['tickformat'] = "3f"

    iplot(fig1)
    
interact(plot_lightcurve,mag_range_select=widgets.FloatRangeSlider(
    min=min(filtered_data['calibrated_magnitude']),max=max(filtered_data['calibrated_magnitude']),
    step=0.01,value=[min(filtered_data['calibrated_magnitude']),max(filtered_data['calibrated_magnitude'])],
    layout={'width': '600px'},description='Magnitude range [mag]',style = {'description_width': 'initial'},continuous_update=False
))

interactive(children=(FloatRangeSlider(value=(11.6636, 19.0543), continuous_update=False, description='Magnitu…

<function __main__.plot_lightcurve(mag_range_select=[11.6636, 19.0543], error_bars=False)>

### Colour curve plot

In [13]:
def view_colourcurve(col1='V',col2='I',error_bars=False):
    jd=Time(filtered_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd

    if error_bars==True:
        fig1=px.scatter(filtered_data,x=mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'},
                    error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])))
    else:
        fig1=px.scatter(filtered_data,x=mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'})
    fig1['layout']['xaxis']['tickformat'] = "3f"

    iplot(fig1)

interact(view_colourcurve,col1= widgets.Dropdown(options=sorted(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=sorted(band),value='I',description='Select second colour',style = {'description_width': 'initial'}))


interactive(children=(Dropdown(description='Select first colour', index=3, options=('B', 'I', 'R', 'V'), style…

<function __main__.view_colourcurve(col1='V', col2='I', error_bars=False)>

### Colour-magnitude plot

In [14]:
def plot_cmd(m1='V',col1='V',col2='I',error_bars=False):
    jd=Time(filtered_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd,
                   error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])),error_x=filtered_data[m1+'e'])
    else:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd)

    x = numpy.linspace(-1, 0, 100)
    y = 3.1 * x +numpy.max(filtered_data[m1])
    fig2=px.line(x=x+numpy.max(filtered_data[col1]-filtered_data[col2]),y=y,labels='R_v=3.1')
    fig2.update_traces(line=dict(color = 'blue',width=3))
    
    x = numpy.linspace(-1, 0, 100)
    y = 5 * x +numpy.max(filtered_data[m1])
    fig3=px.line(x=x+numpy.max(filtered_data[col1]-filtered_data[col2]),y=y,labels='R_v=5')
    fig3.update_traces(line=dict(color = 'red',width=3))
    
    fig4 = go.Figure(data=fig2.data + fig3.data + fig1.data)
    fig4['layout']['yaxis_title']=f'Magnitude ({m1}) [mag]'
    fig4['layout']['xaxis_title']=f'Colour ({col1}-{col2}) [mag]'
    fig4['layout']['yaxis']['autorange'] = "reversed"
    fig4['layout']['coloraxis_colorbar']['title'] = 'Date [mjd]'
    fig4.update_layout(autosize=False,width=900,height=600)
    
    fig4.show()
    #iplot(fig1)
    
interact(plot_cmd,m1= widgets.Dropdown(options=sorted(band),value='V',description='Select magnitude',style = {'description_width': 'initial'}),
                         col1= widgets.Dropdown(options=sorted(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=sorted(band),value='I',description='Select second colour',style = {'description_width': 'initial'}))
   

interactive(children=(Dropdown(description='Select magnitude', index=3, options=('B', 'I', 'R', 'V'), style=De…

<function __main__.plot_cmd(m1='V', col1='V', col2='I', error_bars=False)>